In [1]:
import requests
from html.parser import HTMLParser
from IPython.display import display
import re

In [2]:
emergency_data_url = "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=2%2CHousehold+Event+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Emergency+Room+Visits"

emergency_page = requests.get(emergency_data_url).text

In [15]:
class MepsScraper(HTMLParser):
    p = re.compile('HC-[0-9]*[A-Z]?')
    pufs = []
    
    def handle_data(self, data):
        
        if self.p.match(data):
            self.pufs.append(data)
        
meps = MepsScraper()
meps.feed(emergency_page)
display(meps.pufs)

['HC-160E',
 'HC-152E',
 'HC-144E',
 'HC-135E',
 'HC-126E',
 'HC-118E',
 'HC-110E',
 'HC-102E',
 'HC-094E',
 'HC-085E',
 'HC-077E',
 'HC-067E',
 'HC-059E',
 'HC-051E',
 'HC-033E',
 'HC-026E',
 'HC-016E',
 'HC-010E']

In [4]:
def norm_puf(puf):
    splits = puf.split("C-")
    if splits[1][0] == '0':
        return ''.join([splits[0], splits[1][1:]]).lower()
    else:
        return ''.join(splits).lower()
    
final_pufs = list(map(norm_puf, meps.pufs))
display(final_pufs)

['h160e',
 'h152e',
 'h144e',
 'h135e',
 'h126e',
 'h118e',
 'h110e',
 'h102e',
 'h94e',
 'h85e',
 'h77e',
 'h67e',
 'h59e',
 'h51e',
 'h33e',
 'h26e',
 'h16e',
 'h10e']

In [5]:
puf_url = lambda x: 'http://meps.ahrq.gov/mepsweb/data_files/pufs/' + x + 'ssp.zip'

puf_urls = list(map(puf_url, final_pufs))

puf_files = {puf: requests.get(puf_url(puf)) for puf in final_pufs}

In [6]:
from zipfile import ZipFile, BadZipFile
from io import BytesIO

for key, value in puf_files.items():
    try:
        puf_zip = ZipFile(BytesIO(value.content))
        puf_zip.extractall()
    except BadZipFile:
        display("Error extracting {}".format(key))

'Error extracting h26e'

'Error extracting h16e'

'Error extracting h10e'